In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
# get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np
import pandas as pd
import pickle
import scipy.signal
import glob
import cv2
from skimage.exposure import rescale_intensity
import argparse
from sklearn.cluster import KMeans
from itertools import groupby
from skimage import color, data, restoration
import h5py
import random
from tqdm import tqdm
import os
import time
from patchify import patchify, unpatchify
import glob

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [ ]:
fs = 500000
spec_params={
'nperseg': 512, # default 1024
'noverlap': 256, # default: nperseg / 4
'fs': fs, # raw signal sample rate is 4MHz
'window': 'hamm',
'scaling': 'density', # {'density', 'spectrum'}
'detrend': 'linear', # {'linear', 'constant', False}
'eps': 1e-11}

# returns spectrogram
def specgr (fname,ecen,spec_params,cut_shot):
    ece_data = pickle.load(open(fname,'rb'))
    ece_num = '\\tecef%.2i' % (ecen)
    sig_in = ece_data[ece_num][:np.int_(cut_shot*spec_params['fs'])]
    f, t, Sxx = scipy.signal.spectrogram(sig_in, nperseg=spec_params['nperseg'], noverlap=spec_params['noverlap'],fs=spec_params['fs'], window=spec_params['window'],scaling=spec_params['scaling'], detrend=spec_params['detrend'])
    Sxx = np.log(Sxx + spec_params['eps'])
    Sxx=(Sxx-np.min(Sxx))/(np.max(Sxx)-np.min(Sxx))
    Sxx = Sxx[:-1,:];f=f[:-1]
    return Sxx,f,t

def norm(data):
    mn = data.mean()
    std = data.std()
    return((data-mn)/std)

def rescale(data):
    return (data-data.min())/(data.max()-data.min())

def quantfilt(src,thr=0.9):
    filt = np.quantile(src,thr,axis=0)
    out = np.where(src<filt,0,src)
    return out

# gaussian filtering
def gaussblr(src,filt=(31, 3)):
    src = (rescale(src)*255).astype('uint8')
    out = cv2.GaussianBlur(src,filt,0)
    return rescale(out)

# mean filtering
def meansub(src):
    mn = np.mean(src,axis=1)[:,np.newaxis]
    out = np.absolute(src - mn)
    return rescale(out)

# morphological filtering
def morph(src):
    src = (rescale(src)*255).astype('uint8')
    
    se1 = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
    se2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
    mask = cv2.morphologyEx(src, cv2.MORPH_CLOSE, se1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, se2)
        
    return rescale(mask)

#reshapes the data
def reshape(arr):
    arr = np.reshape(arr, (len(arr), 256, 128, 1))
    return arr

# patches all the strips together to 1 spectrogram
def patch(arr):
    all_patches = np.empty((len(arr)* 30, 256, 128))
    for i in range(len(arr)):
        patches = patchify(arr[i], (256, 128), step=128)
        
        for x in range(30):
            all_patches[(x + 30 * i)] = patches[0][x]
    
    return all_patches

# splits spectrogram into strips
def unpatch(arr):
    all_spectrograms = []
    for i in range(int(len(arr) / 30)):
        Sxx = []
        for x in range(30):
            Sxx.append(arr[x + 30 * i])
        
        y=[Sxx]
        reconstructed = unpatchify(np.array(y), (256, 3840))
        all_spectrograms.append(reconstructed)
    return np.array(all_spectrograms)

# displays Sxx and final
def display(Sxx, final):
    n = 5
    
    idx = np.random.randint(len(Sxx), size=n)
    plots1 = Sxx[idx, :]
    plots2 = final[idx, :]
    
    fig = plt.figure(figsize=(8,12))
    grd = gridspec.GridSpec(ncols=1, nrows=(2 * n), figure=fig)
    ax=[None] * (2 * n)
    
    for i, (plot1, plot2) in enumerate(zip(plots1, plots2)):
        ax[2*i] = fig.add_subplot(grd[2*i])
        ax[2*i].pcolormesh(t[:3840],(f/1000)+1,plot1,cmap='hot',shading='gouraud')
        _=plt.ylabel('Original')
        
        ax[2*i+1] = fig.add_subplot(grd[2*i+1])
        ax[2*i+1].pcolormesh(t[:3840],(f/1000)+1,plot2,cmap='hot',shading='gouraud')
        _=plt.ylabel('Final')
    
    plt.show()

In [ ]:
# load data
data_path = "/scratch/gpfs/aonelson/ml_database/ECE_data/"
flist = glob.glob(data_path+"*.pkl")
result_path = "frames/"

spectrograms = []
final = []
thr = 0.9

# randomly picks 100 pkl files and adds original and processed spectrograms to separate lists

random_sample = random.sample(flist, 100)

for fname in tqdm(random_sample):
    shotn = fname[fname.rfind('_')+1:fname.rfind('.')]

    for chn in range(20):
        try:
            s,f,t = specgr(fname,chn+1,spec_params,2)
            
            # image processing pipeline
            out_quant= quantfilt(s,thr)
            out_gauss=np.empty(s.shape)
            out_mean=np.empty(s.shape)
            out_morph = np.empty(s.shape)
            out_final = np.empty(s.shape)
            
            out_gauss =  gaussblr(out_quant,(31, 3))
            out_mean = meansub(out_gauss)    
            out_morph = morph(out_mean)
            out_final = meansub(out_morph)
            
            spectrograms.append(s)        
            final.append(out_final)
            
        except pickle.UnpicklingError as e:
            continue
        except Exception as e:
            print(traceback.format_exc(e))
            continue

In [ ]:
# splits into strips
spectrograms = patch(spectrograms)
final = patch(final)

#split into 60% (train), 25% (tune), 15% (test)
Sxx_train, Sxx_tune, Sxx_test = np.split(spectrograms, [int(len(spectrograms)*0.6), int(len(spectrograms)*0.85)])
final_train, final_tune, final_test = np.split(final, [int(len(final)*0.6), int(len(final)*0.85)])

print(Sxx_train.shape, Sxx_tune.shape, Sxx_test.shape)
print(final_train.shape, final_tune.shape, final_test.shape)

In [ ]:
display(unpatch(Sxx_test), unpatch(final_test))

In [ ]:
input = layers.Input(shape = (256, 128, 1))

x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)

x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")
autoencoder.summary()

In [ ]:
# reshape our data
Sxx_train_reshaped = reshape(Sxx_train)
Sxx_test_reshaped = reshape(Sxx_test)
Sxx_tune_reshaped = reshape(Sxx_tune)
final_train_reshaped = reshape(final_train)
final_test_reshaped = reshape(final_test)
final_tune_reshaped = reshape(final_tune)

In [ ]:
autoencoder.fit(
    x=Sxx_train_reshaped,
    y=final_train_reshaped,
    epochs=100,
    batch_size=128,
    shuffle=True,
    validation_data=(Sxx_tune_reshaped, final_tune_reshaped),
)

In [ ]:
# predict and reformat
predictions = autoencoder.predict(Sxx_test_reshaped)
predictions = np.squeeze(predictions, axis=3)

In [ ]:
# restitch everything together to a list of spectrograms

noisy = unpatch(Sxx_test)
autoencoder_final = unpatch(predictions)
pipeline_final = unpatch(final_test)

In [ ]:
fig = plt.figure(figsize=(8,12))
grd = gridspec.GridSpec(ncols=1, nrows=3, figure=fig)
ax=[None] * 3

ax[0] = fig.add_subplot(grd[0])
ax[0].pcolormesh(t[:3840],(f/1000)+1,noisy[0],cmap='hot',shading='gouraud')
_=plt.ylabel('Original')

ax[0] = fig.add_subplot(grd[1])
ax[0].pcolormesh(t[:3840],(f/1000)+1,autoencoder_final[0],cmap='hot',shading='gouraud')
_=plt.ylabel('Autoencoder Denoised')

ax[2] = fig.add_subplot(grd[2])
ax[2].pcolormesh(t[:3840],(f/1000)+1,pipeline_final[0],cmap='hot',shading='gouraud')
_=plt.ylabel('Pipeline Denoised')

plt.show()

In [ ]:
display(noisy, autoencoder_final)

In [ ]:
# testing with file 176053

data_path = "/scratch/gpfs/aonelson/ml_database/ECE_data/"
flist = glob.glob(data_path+"*.pkl")
fname = '/scratch/gpfs/aonelson/ml_database/ECE_data/ece_176053.pkl'

shotn = fname[fname.rfind('_')+1:fname.rfind('.')]

Sxx = []
processed = []
thr = 0.9

for chn in range(40):
    s,f,t = specgr(fname,chn+1,spec_params,2)
    
    out_quant= quantfilt(s,thr)
    out_gauss=np.empty(s.shape)
    out_mean=np.empty(s.shape)
    out_morph = np.empty(s.shape)
    out_final = np.empty(s.shape)

    out_gauss =  gaussblr(out_quant,(31, 3))
    out_mean = meansub(out_gauss)    
    out_morph = morph(out_mean)
    out_final = meansub(out_morph)
    
    Sxx.append(s)
    processed.append(out_final)

In [ ]:
noisy = patch(Sxx)
processed = patch(processed)
predictions = autoencoder.predict(reshape(noisy))
predictions = np.squeeze(predictions, axis=3)

In [ ]:
noisy = unpatch(noisy)
processed = unpatch(processed)
predictions = unpatch(predictions)

In [ ]:
# Shot 170670 -> channels 10-19
for i in range(10):
    fig = plt.figure(figsize=(8,12))
    grd = gridspec.GridSpec(ncols=1, nrows=4, figure=fig)
    ax=[None] * 4
    caption = 'shot# %s, channel %i' % (shotn, i + 10)
    
    ax[0] = fig.add_subplot(grd[0])
    ax[0].pcolormesh(t[:3840],(f/1000)+1,noisy[i + 10],cmap='hot',shading='gouraud')
    _=plt.ylabel('Original')
    ax[0].set(title=caption)

    ax[1] = fig.add_subplot(grd[1])
    ax[1].pcolormesh(t[:3840],(f/1000)+1,predictions[i + 10],cmap='hot',shading='gouraud')
    _=plt.ylabel('Predicted Denoised')

    plt.show()

In [ ]:
# stack all the spectrograms back together into matrix
noisy = np.dstack(noisy)
processed = np.dstack(processed)
predictions = np.dstack(predictions)
print(noisy.shape, processed.shape, predictions.shape)

In [ ]:
frm = 900

fig = plt.figure(figsize=(9,6))
grd = gridspec.GridSpec(ncols=1, nrows=3, figure=fig)
ax=[None] * 3
caption = 'shot# %s, fr# %i/%i, t:%ims' % (shotn, frm, noisy.shape[1], t[frm]*1000)
ax[0] = fig.add_subplot(grd[0])
ax[0].cla()
ax[0].pcolormesh(range(40),(f/1000)+1,noisy[:,frm,:],cmap='hot',shading='gouraud')
ax[0].set(ylabel='Freq. (KHz)',yscale='linear',xticks=[],title=caption)

ax[1] = fig.add_subplot(grd[1])
ax[1].cla()
ax[1].pcolormesh(range(40),(f/1000)+1,processed[:,frm,:],cmap='hot',shading='gouraud')
ax[1].set(ylabel='Freq. (KHz)',yscale='linear',xticks=[])

ax[2] = fig.add_subplot(grd[2])
ax[2].cla()
ax[2].pcolormesh(range(40),(f/1000)+1,predictions[:,frm,:],cmap='hot',shading='gouraud')
ax[2].set(ylabel='Freq. (KHz)',yscale='linear',xlabel='ECE Channel')

plt.show()

In [ ]:
# Saves frames
for i in tqdm(range(2569, len(noisy[0]))):
    fig = plt.figure(figsize=(9,6))
    grd = gridspec.GridSpec(ncols=1, nrows=3, figure=fig)
    ax=[None] * 3
    caption = 'shot# %s, fr# %i/%i, t:%ims' % (shotn, i+1, noisy.shape[1], t[i]*1000)
    ax[0] = fig.add_subplot(grd[0])
    ax[0].cla()
    ax[0].pcolormesh(range(40),(f/1000)+1,noisy[:,i,:],cmap='hot',shading='gouraud')
    ax[0].set(ylabel='Freq. (KHz)',yscale='linear',xticks=[],title=caption)
    
    ax[1] = fig.add_subplot(grd[1])
    ax[1].cla()
    ax[1].pcolormesh(range(40),(f/1000)+1,processed[:,i,:],cmap='hot',shading='gouraud')
    ax[1].set(ylabel='Freq. (KHz)',yscale='linear',xticks=[])
    
    ax[2] = fig.add_subplot(grd[2])
    ax[2].cla()
    ax[2].pcolormesh(range(40),(f/1000)+1,predictions[:,i,:],cmap='hot',shading='gouraud')
    ax[2].set(ylabel='Freq. (KHz)',yscale='linear',xlabel='ECE Channel')
    plt.savefig('frames/s' + str(shotn) + '-f' + str(i).zfill(5) + '.jpg')
    
    plt.close()

In [ ]:
# Stitches frames together
shotn = 176053
result_path = 'frames/' # Where the frames are stored as sXXX-fYYYYY.jpg (XXX is shot number and YYYYY is frame number beginning from 00000)
frmlist = glob.glob(result_path+'*.jpg') # Get the list of frames
# read first file to get size
img = cv2.imread(frmlist[0])
height, width, layers = img.shape
size = (width,height)
fname_movie_out = result_path+str(shotn)+'.mp4'
out = cv2.VideoWriter(fname_movie_out ,cv2.VideoWriter_fourcc(*'H264'), 30, size)
for frm in tqdm(range(len(frmlist)), desc='Making movie...', ascii=True, dynamic_ncols=True):
    filename = '%ss%s-f%.5i.jpg' %(result_path,shotn,frm)
    img = cv2.imread(filename)
    out.write(img)
out.release()